**Кулешов Иван AML-14**

# Домашнее задание по теме «Коллаборативная фильтрация»



Задание:\
ПАКЕТ SURPRISE:
 - используйте данные MovieLens 1M
 - можно использовать любые модели из пакета
 - получите RMSE на тестовом сете 0.87 и ниже

## Загружаем данные

Для моделирования используется инструмент Google Colab:

In [ ]:
# scikit-surprise-1.1.1
!pip install scikit-surprise

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, SVD, SVDpp
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Скачиваем данные для датасета ml-latest mini, на полном датасете ноутбук не запустился :(

In [2]:
path = "/content/drive/MyDrive/Netology-homework/3 RSML Рекомендательные системы/data-hw1"

movies = pd.read_csv(path + '/movies.csv')
ratings = pd.read_csv(path + '/ratings.csv', usecols=['userId', 'movieId', 'rating'])

Объединяем данные о фильмах и данные о рейтинге:

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [4]:
movies_with_ratings.head()


,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5


Для пакета SURPRISE должна выдерживаться структура датафрейма: три колонки, - user, item, rating.

In [5]:
df = movies_with_ratings[['userId', 'title', 'rating']]

Посмотрим разброс по рейтингу (0,5 - 5,0). Эти значения используем для задания разброса шкалы для класса Reader.

In [6]:
set(movies_with_ratings.rating)

{0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0}

In [7]:
reader = Reader(rating_scale=(0.5, 5)) # разброс по рейтингу
dataset = Dataset.load_from_df(df, reader) # читаем датасет из pd.DataFrame

## Моделирование

### Базовая модель

In [33]:
X_train, X_test = train_test_split(dataset, test_size=0.2)

In [9]:
model = KNNBasic(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
model.fit(X_train)
predictions = model.test(X_test)
print(f'RMSE on TEST 20% = {accuracy.rmse(predictions)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9636
RMSE on TEST 20% = 0.9636487609090657


Как очевидно, по результату RMSE мы пока своей цели не достигли. Попробуем другие модели:

### Другие модели SURPRISE

Алгоритм, учитывающий средние оценки каждого пользователя.

In [10]:
model = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
cross_validate(model, dataset, measures=['RMSE'], cv=5, verbose=False)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.8851749 , 0.88131631, 0.88060489, 0.88760522, 0.86896625]),
 'fit_time': (17.546558380126953,
  17.6558039188385,
  14.40025806427002,
  14.453295707702637,
  13.892237186431885),
 'test_time': (11.134957075119019,
  10.383501052856445,
  10.513562202453613,
  10.669912576675415,
  9.441960573196411)}

Алгоритм, основанный на сингулярном разложении

In [11]:
model = SVD()
cross_validate(model, dataset, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8778  0.8817  0.8755  0.8655  0.8685  0.8738  0.0060  
Fit time          5.93    5.92    5.92    5.89    5.75    5.88    0.07    
Test time         0.34    0.16    0.16    0.38    0.17    0.24    0.10    


{'test_rmse': array([0.8778186 , 0.88174361, 0.87550936, 0.8654669 , 0.86850889]),
 'fit_time': (5.9302756786346436,
  5.916168928146362,
  5.919912815093994,
  5.8876793384552,
  5.746811628341675),
 'test_time': (0.3419454097747803,
  0.15827131271362305,
  0.1594250202178955,
  0.38243794441223145,
  0.17109370231628418)}

Расширенный алгоритм сингулярного разложения с учетом неявных рейтингов

In [34]:
model = SVDpp()
model.fit(X_train)
predictions = model.test(X_test)
print(f'RMSE on TEST 20% = {accuracy.rmse(predictions)}')

RMSE: 0.8569
RMSE on TEST 20% = 0.856898852453381


Эта модель повела себя успешнее других, хотя и очень долго считается. Однако, поставленная задачей точность rmse < 0.87 достигнута. Посмотрим, удастся ли уменьшить метрику путём тюнинга гиперпараметров для модели SVD:

In [12]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [100, 200], 'n_epochs': [50, 100, 200], 'lr_all': [0.002, 0.005],
              'biased': [True, False], 'reg_all': [0.02, 0.1, 0.2], 'verbose': [True]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1, joblib_verbose=True)
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 91.5min


0.8481259141362292
{'n_factors': 200, 'n_epochs': 200, 'lr_all': 0.005, 'biased': True, 'reg_all': 0.1, 'verbose': True}


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 229.8min finished


Получилось, значение RMSE стало ещё меньше.

## Построение рекомендаций

Наша основная задача всего моделирования - получить в итоге систему рекомендации. Реализуем её так:
- для указанного пользователя возьмём фильмы, которые он не оценивал (то есть, не смотрел), пропустим эти фильмы через нашу модель предсказав оценку, которую поставил бы фильмам пользователь, и фильмы с наивысшим предсказанным рейтингом порекомендуем ему.

### Формирование датафрейма для предикта

In [14]:
TARGET_USER = 474

Выберем все фильмы, которые пользователь не оценивал и для каждого фильма оставим только одну строчку:

In [15]:
df_for_predict = movies_with_ratings[movies_with_ratings['userId']!=TARGET_USER].drop_duplicates(subset='movieId')

Выбираем колонки, необходимые для работы модели.

In [16]:
df = df_for_predict[['userId', 'title', 'rating']]

In [ ]:
df['userId'] = TARGET_USER

С помощью трюка с test_size разбиваем датасет так, чтобы вся тестовая выборка равнялась нашему датасету (связано с разностью внутренних форматов пакета surprise - датасет имеет формат DatasetAutoFolds, и на нём нельзя обучать и получать предикт

In [72]:
predict_dataset = Dataset.load_from_df(df, reader)
_, X_predict = train_test_split(predict_dataset, test_size=1.0, shuffle=False)

Обучаем модель, используя лучшие гиперпараметры:

In [73]:
model_best = SVD(n_factors=200, n_epochs=200, lr_all=0.005, biased=True, reg_all=0.1)
model_train = model_best.fit(X_train)

Получаем предсказанные рейтинги фильмов, которые юзер не смотрел, следующим образом:

In [82]:
predict =[]

for _,item in df.iterrows():
  pr = model_train.predict(uid=TARGET_USER, iid=item['title'])

  row = {}
  row['title'] = pr.iid
  row['predicted_rating'] = pr.est
  predict.append(row)

df_for_predict = pd.DataFrame.from_dict(predict)

In [83]:
df_for_predict

,title,predicted_rating
0,Toy Story (1995),3.654542
1,Jumanji (1995),3.308958
2,Grumpier Old Men (1995),2.876066
3,Waiting to Exhale (1995),2.579048
4,Father of the Bride Part II (1995),2.652804
...,...,...
9458,Black Butler: Book of the Atlantic (2017),3.520544
9459,No Game No Life: Zero (2017),3.479747
9460,Flint (2017),3.478953
9461,Bungo Stray Dogs: Dead Apple (2018),3.335682


Или таким образом:

In [74]:
predicts = model_train.test(X_predict)

predict = [p.est for p in predicts]

In [75]:
df_for_predict['predicted_rating'] = predict

In [84]:
df_for_predict

,title,predicted_rating
0,Toy Story (1995),3.654542
1,Jumanji (1995),3.308958
2,Grumpier Old Men (1995),2.876066
3,Waiting to Exhale (1995),2.579048
4,Father of the Bride Part II (1995),2.652804
...,...,...
9458,Black Butler: Book of the Atlantic (2017),3.520544
9459,No Game No Life: Zero (2017),3.479747
9460,Flint (2017),3.478953
9461,Bungo Stray Dogs: Dead Apple (2018),3.335682


Ну и наконец отсортируем по рейтингу и выдадим результат рекомедации Топ-20:

In [85]:
df_for_predict.sort_values(by='predicted_rating', ascending=False)[['title', 'predicted_rating']][:20]

,title,predicted_rating
1681,"Impostors, The (1998)",4.651035
4625,Adam's Rib (1949),4.576722
5849,Sophie Scholl: The Final Days (Sophie Scholl -...,4.544173
870,Enchanted April (1992),4.543880
4940,Safety Last! (1923),4.537848
4441,Babette's Feast (Babettes gæstebud) (1987),4.501115
4830,Pierrot le fou (1965),4.415094
9339,"Three Billboards Outside Ebbing, Missouri (2017)",4.397450
2543,Guess Who's Coming to Dinner (1967),4.395865
5582,"Wild Parrots of Telegraph Hill, The (2003)",4.391842
